In [21]:
from spineq.data_fetcher import get_traffic_counts
from spineq.weights import calc_location_weights

import contextily as ctx
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
traffic_counts = get_traffic_counts()
traffic_counts.head()

In [20]:
traffic_counts["traffic_co"].plot.hist(bins=20)

traffic_counts["traffic_co"].describe()

In [18]:
fig, ax = plt.subplots(1,1, figsize=(10,10))
markersize = 200*traffic_counts["traffic_co"]/max(traffic_counts["traffic_co"])
traffic_counts.sort_values(by="traffic_co").plot(
    ax=ax,
    markersize=markersize,
    color="r"
)

ctx.add_basemap(
    ax,
    source="http://a.tile.stamen.com/toner/{z}/{x}/{y}.png",
    crs=traffic_counts.crs.to_epsg()
)

## Traffic intersection counts only optimisation

In [22]:
weights = calc_location_weights(
    population_weight=1,
    workplace_weight=0,
    pop_age_groups={
        "pop_total": {"min": 0, "max": 90, "weight": 1},
        "pop_children": {"min": 0, "max": 16, "weight": 1},
        "pop_elderly": {"min": 70, "max": 90, "weight": 0},
    },
    traffic_weight=1,
    combine=False,
)

In [10]:
traffic_counts = get_traffic_counts()["traffic_co"]
traffic_weight = traffic_counts / traffic_counts.sum()
traffic_weight.name = "traffic"

In [11]:
oa_weight = calc_location_weights(workplace_weight=1, combine=False)
oa_weight

In [12]:
all_weight = oa_weight.join(traffic_weight, how="outer").fillna(0)
all_weight

In [13]:
oa_weight["poi_type"] = "oa"

traffic_weight = pd.DataFrame(traffic_weight)
traffic_weight["poi_type"] = "intn"

In [14]:
all_weight["poi_type"] = np.nan
all_weight.loc[all_weight.index.isin(oa_weight.index), "poi_type"] = "output_area"
all_weight.loc[all_weight.index.isin(traffic_weight.index), "poi_type"] = "intersection"

all_weight